In [ ]:
import numpy as np
from scipy.stats import pearsonr, t
from itertools import combinations
from linear_regression import LinearRegression


# Läs in data från csv-filen
#---------------------------------------------

data = np.genfromtxt(
    "data/housing.csv",
    delimiter=",",
    skip_header=1,
    usecols=range(9)
)

data = data[~np.isnan(data).any(axis=1)]

Y = data[:, -1]
X_all = data[:, :-1]

feature_names = [
    "longitude",
    "latitude",
    "housing_median_age",
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income"
]

print("Antal observationer:", len(Y))


# Sortera features efter korrelation
#---------------------------------------------

print("\nFeatures sorterade efter |korrelation|:\n")

corr_results = []

for i in range(X_all.shape[1]):
    r, p = pearsonr(X_all[:, i], Y)
    corr_results.append((feature_names[i], r, p))

corr_results_sorted = sorted(
    corr_results,
    key=lambda x: abs(x[1]),
    reverse=True
)

for name, r, p in corr_results_sorted:
    print(f"{name:20s} r = {r:.3f}  p = {p:.5f}")


# Automatiskt val av feature
#---------------------------------------------

def select_best_features(X, Y, names, max_d=4, corr_threshold=0.85):

    best_r2 = -np.inf
    best_combo = None
    best_model = None

    for d in range(1, max_d + 1):
        for combo in combinations(range(X.shape[1]), d):

            high_corr = False
            for i in range(len(combo)):
                for j in range(i + 1, len(combo)):
                    r, _ = pearsonr(X[:, combo[i]], X[:, combo[j]])
                    if abs(r) > corr_threshold:
                        high_corr = True
                        break
                if high_corr:
                    break

            if high_corr:
                continue

            X_subset = X[:, combo]
            model = LinearRegression()
            model.fit(X_subset, Y)

            r2 = model.r_squared()

            if r2 > best_r2:
                best_r2 = r2
                best_combo = combo
                best_model = model

    print("\nBästa features:")
    for i in best_combo:
        print("-", names[i])

    print("Bästa R²:", round(best_r2, 4))

    return best_combo, best_model


best_combo, model = select_best_features(
    X_all,
    Y,
    feature_names,
    max_d=4
)

selected_names = [feature_names[i] for i in best_combo]



# Sammanfattning
#---------------------------------------------

def print_summary(model, feature_names, alpha=0.05):

    print("\n" + "="*75)
    print("MULTIPEL LINJÄR REGRESSION – MODELLSAMMANFATTNING")
    print("="*75)

    print(f"Antal observationer (n): {model.n}")
    print(f"Antal features (d): {model.d}")
    print(f"R²: {model.r_squared():.4f}")
    print(f"RMSE: {model.rmse():.2f}")

    F_stat, F_p = model.f_test()
    print(f"F-statistik: {F_stat:.3f}")
    print(f"F-test p-värde: {F_p:.5f}")

    print("\n" + "-"*75)
    print("Koefficienter:")
    print("-"*75)

    t_vals, p_vals = model.t_tests()

    df = model.n - model.d - 1
    t_crit = t.ppf(1 - alpha/2, df)

    header = f"{'Variabel':20s} {'Beta':>10s} {'Std.fel':>10s} {'t':>10s} {'p':>10s} {'CI lower':>12s} {'CI upper':>12s}"
    print(header)
    print("-"*len(header))

    names = ["Intercept"] + feature_names

    for i in range(len(model.beta)):
        beta = model.beta[i].item()
        se = np.sqrt(model.cov_beta[i, i])
        t_stat = t_vals[i]
        p_val = p_vals[i]

        lower = beta - t_crit * se
        upper = beta + t_crit * se

        print(f"{names[i]:20s} "
              f"{beta:10.2f} "
              f"{se:10.2f} "
              f"{t_stat:10.2f} "
              f"{p_val:10.5f} "
              f"{lower:12.2f} "
              f"{upper:12.2f}")

    print("="*75)


print_summary(model, selected_names)


# Scenarioanalys
# Framtida händelsers inverkan på bostadspriset
#---------------------------------------------

if "median_income" in selected_names:

    income_index = selected_names.index("median_income")
    beta_income = model.beta[income_index + 1].item()

    delta_income = 0.5
    price_change = beta_income * delta_income

    print("\nScenarioanalys:")
    print("Om median_income ökar med 0.5:")
    print("Prisförändring ≈", round(price_change, 2))


# Elasticitet
# Bostadspriser utifrån yttre faktorer
#---------------------------------------------

if "median_income" in selected_names:

    X_selected = X_all[:, best_combo]
    mean_income = np.mean(X_selected[:, income_index])
    mean_price = np.mean(Y)

    elasticity = beta_income * (mean_income / mean_price)

    print("\nElasticitet:")
    print("Elasticitet =", round(elasticity, 3))


# Konfidensintervall
# Medelpris för en genomsnittlig bostad
#---------------------------------------------

X_selected = X_all[:, best_combo]
x0 = np.mean(X_selected, axis=0)
x0 = np.insert(x0, 0, 1).reshape(-1, 1)

XtX_inv = np.linalg.pinv(model.X.T @ model.X)

var_pred = model.sigma2_hat * (x0.T @ XtX_inv @ x0)
se_pred = np.sqrt(var_pred)

df = model.n - model.d - 1
t_crit = t.ppf(0.975, df)

pred_mean = (x0.T @ model.beta).item()

lower = (pred_mean - t_crit * se_pred).item()
upper = (pred_mean + t_crit * se_pred).item()

print("\n95% Konfidensintervall (genomsnittlig bostad):")
print(round(lower, 2), "till", round(upper, 2))


Antal observationer: 20433

Features sorterade efter |korrelation|:

median_income        r = 0.688  p = 0.00000
latitude             r = -0.145  p = 0.00000
total_rooms          r = 0.133  p = 0.00000
housing_median_age   r = 0.106  p = 0.00000
households           r = 0.065  p = 0.00000
total_bedrooms       r = 0.050  p = 0.00000
longitude            r = -0.045  p = 0.00000
population           r = -0.025  p = 0.00030

Bästa features:
- latitude
- housing_median_age
- total_bedrooms
- median_income
Bästa R²: 0.5315

MULTIPEL LINJÄR REGRESSION – MODELLSAMMANFATTNING
Antal observationer (n): 20433
Antal features (d): 4
R²: 0.5315
RMSE: 79014.53
F-statistik: 5792.604
F-test p-värde: 0.00000

---------------------------------------------------------------------------
Koefficienter:
---------------------------------------------------------------------------
Variabel                   Beta    Std.fel          t          p     CI lower     CI upper
------------------------------------------